# 1. Neural Network Basics

In [ ]:
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

np.random.seed(10)

(ignore warning messages if you see any)

## 1.1 How a Neural Network Makes Predictions

In [ ]:
# example input/response mapping 
def f(x):
    return np.where(np.dot(x, np.random.rand(3, 1)) > 0.8, 1., 0.)
    
X = np.random.random((1000, 3))
y_true = f(X)

# print(X[:3])
# print(y_true[:3])

Network topology:

<img src="images/Network.png"/>

In [ ]:
# create the model
neural_network = Sequential()

In [ ]:
hidden_layer1 = Dense(units=2, activation="relu", input_dim=3)
neural_network.add(hidden_layer1)

In [ ]:
hidden_layer2 = Dense(units=3, activation='relu')
neural_network.add(hidden_layer2)
output_layer = Dense(units=2, activation="sigmoid")
neural_network.add(output_layer)

In [ ]:
# summarize model architecture
neural_network.summary()

In [ ]:
# let's manually set the network's weights 
# (input neuron weights, bias weights)

## input --> hidden layer 1
random_weights1 = [np.random.random((3, 2)), np.ones(2)]
neural_network.layers[0].set_weights(random_weights1)

## hidden layer 1 --> hidden layer 2
random_weight2 = [np.random.random((2, 3)), np.ones(3)]
neural_network.layers[1].set_weights(random_weight2)

## hidden layer 2 --> output layer
random_weight3 = [np.random.random((3, 2)), np.ones(2)]
neural_network.layers[2].set_weights(random_weight3)

In [ ]:
# predict y from a few X's
X_sample = X[:10,:]
y_sample_pred = neural_network.predict_classes(X_sample)
y_sample_true = y_true[:10]
print("Pred y:\t{}".format(y_sample_pred.reshape(-1)))
print("True y:\t{}".format(y_sample_true.reshape(-1)))

In [ ]:
# evaluate model on predictive accuracy for all X
y_pred = neural_network.predict_classes(X)
acc = sum(y_true == y_pred.reshape(-1))[0]/float(y_true.shape[0])*100
print("Accuracy: {}%".format(acc))

## 1.2. How to Train a Neural Network

Network topology:

<img src="images/Network.png"/>

In [ ]:
# create model
smart_neural_network = Sequential()

smart_neural_network.add(Dense(2, activation='relu', input_dim=3))
smart_neural_network.add(Dense(3, activation='relu'))
smart_neural_network.add(Dense(2, activation='sigmoid'))

In [ ]:
# define training parameters
smart_neural_network.compile(metrics=['accuracy'],
                             loss='binary_crossentropy',
                             optimizer='rmsprop')

In [ ]:
# pre-processing: convert responses to 2d categorical vectors
from keras.utils import np_utils
y_categorical = np_utils.to_categorical(y_true)

<img src="images/GradientDescent.png" width=400/>

In [ ]:
# train the model for a number of rounds
intial_weights = smart_neural_network.get_weights()
smart_neural_network.fit(X, y_categorical, 
                         epochs=100,
                         batch_size=32
)

In [ ]:
print(smart_neural_network.metrics_names)

In [ ]:
# evaluate the model accuracy on training data
score = smart_neural_network.evaluate(X, y_categorical)
print('Training accuracy: {}%'.format(score[1]*100))

### But how well can our neural network do on data it's never seen before?

In [ ]:
# split data into training and validation sets
NUM_TRAINING_EXAMPLES = 800

X_train = X[:NUM_TRAINING_EXAMPLES]
y_train = y_categorical[:NUM_TRAINING_EXAMPLES]
X_valid = X[NUM_TRAINING_EXAMPLES:]
y_valid = y_categorical[NUM_TRAINING_EXAMPLES:]

In [ ]:
# re-train only on training examples
smart_neural_network.set_weights(intial_weights)

smart_neural_network.compile(metrics=['accuracy'], loss='binary_crossentropy', optimizer='rmsprop')
smart_neural_network.fit(X_train, y_train, batch_size=32, epochs=100)   

In [ ]:
# evaluate on validation (never-seen-before) examples 
validation_scores = smart_neural_network.evaluate(X_valid, y_valid)
print('Validation accuracy: {}%'.format(score[1]*100))

## Exercise: deep networks

Try creating a neural network model with a "deeper" architecture (e.g. more than just 1-2 hidden layers). Fit this on the training data from the input/response example given in the notebook and evaluate the predictive error. 

* Experiment with the number of neurons in each layer.
* When training, increase the number of epochs 

Do you experience any performance gains? Does training run slower than it did before? If so, by how much? Are the performance gains worth it? 

In [ ]:
# TODO

<i>Sample solution:</i>

In [ ]:
# deep_neural_network = Sequential()
# deep_neural_network.add(Dense(128, activation='relu', input_dim=3))
# deep_neural_network.add(Dense(64, activation='relu'))
# deep_neural_network.add(Dense(32, activation='relu'))
# deep_neural_network.add(Dense(2, activation='sigmoid'))
# deep_neural_network.compile(metrics=['accuracy'], loss='binary_crossentropy', optimizer='rmsprop')

# deep_neural_network.fit(X_train, y_train, epochs=100, batch_size=32)
# score = deep_neural_network.evaluate(X_valid, y_valid)

# print 'Predictive accuracy on validation data:', score[1]